# Austin, TX - Real State

## Introduction

Over the past ten years the city of Austin, TX went from being the 14th to the 11th most populous city in the United States. This growth is largely in part to the tech market which has grown by 25% over the past five years, making Austin the home to many tech giants, including Google, Facebook, Oracle, and Apple. This growth has also impacted surrounding areas like New Braunfels, Cedar Park and Round Rock.
As the region grows, it is predicted between 3 and 4 million people will live in the 4 metro areas by 2040. Housing is expected to become more expensive and more compact as the population continues to grow.

## Bussiness Problem

Finding and purchasing a house is already a complicated process that involves a lot of variables. Some of those variables are price, which requires research through different websites and/or real state agents; location, the house should be in a good neighborhood with great gathering spots, parks, shops and a good school district. For this project we will look at real state data from Austin and it's surrounding areas to determine good deals for potential buyers  in this growing regions based on their requirements. 

## Data

The data needed for this project will include:
1. Zillow website data. Using the API from this real state website we can gather information about house prices, location, school districts, etc. 
2. FourSquare data. Using FourSquare API we can get information on the neighborhoods, including shops, restaurants, and venues around them.
3. Wikipedia. Use this website to gather information about the city of Austin and its surrounding areas.